# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re

from random import randint

In [20]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [90]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    def wrapper(*args, **kwargs):
      start_time = time.time()
      result = func(*args, **kwargs)
      end_time = time.time()
      execution_time = end_time - start_time
      print(f'Время выполнения функции {func.__name__}: {execution_time}')
      return result

    return wrapper

In [95]:
# Test 1

@benchmark
def test(a):
  print(a)

test('Hello_world!')

Hello_world!
Время выполнения функции test: 6.198883056640625e-05


## Задание 2

In [80]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    def wrapper(*args, **kwargs):
      print(f"Функция вызвана с параметрами: {args, kwargs}")
      return func(*args, **kwargs)

    return wrapper

In [96]:
# Test 2

@logging
@benchmark
def test(a):
  print(a)

test('Hello_world!')

Функция вызвана с параметрами: (('Hello_world!',), {})
Hello_world!
Время выполнения функции test: 2.6464462280273438e-05


## Задание 3

In [61]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    def wrapper(*args, **kwargs):
        wrapper.count += 1
        print(f"Функция {func.__name__} была вызвана {wrapper.count} раз(а)")
        return func(*args, **kwargs)
    wrapper.count = 0
    return wrapper

In [97]:
# Test 3

@counter
@logging
@benchmark
def test(a):
  print(a)
# print(test.count)

In [93]:
test('Hello_world!')

Функция wrapper была вызвана 1 раз(а)
Функция вызвана с параметрами: (('Hello_world!',), {})
Hello_world!
Время выполнения функции test: 0.00042700767517089844


## Задание 4

In [98]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}

  def fmemo(*args):
    if args in cache:
      return cache[args]
    result = func(*args)
    cache[args] = result
    return result

  # return wrapper
  # fmemo.cache = cache
  return fmemo

## Тестирование

In [94]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция wrapper была вызвана 1 раз(а)
Функция вызвана с параметрами: (('whole',), {})
Время выполнения функции word_count: 0.9243447780609131
Cлово whole встречается 176 раз


In [108]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [113]:
# измеряем время выполнения
%%time
fib(30)

CPU times: user 339 ms, sys: 2.03 ms, total: 341 ms
Wall time: 344 ms


832040

In [114]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [115]:
# измеряем время выполнения
%%time
fib(40)

CPU times: user 71 µs, sys: 1 µs, total: 72 µs
Wall time: 75.8 µs


102334155